# NER 命名实体识别任务baseline
本示例仅提供完整的运行流程，供大家进行初步学习<br>
自行尝试须注意基本配置与文件目录的定义<br>
具体脚本内容可在相应.py文件内查看

### 1.基本配置
安装脚本依赖库<br>
每次重启notebook须重新安装

In [1]:
!pip install docopt
!pip install transformers

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=dd456cdfe2b46b704bb5991672a5fe01c510e728a6a0804012dcb0c6580ec835
  Stored in directory: /home/ma-user/.cache/pip/wheels/6a/96/4d/255bd44721c1c199348fc6c9090b061b587eeaf5a7d6315c08
Successfully built docopt
You should consider upgrading via the '/home/ma-user/anaconda3/envs/PyTorch-1.8/bin/python3.7 -m pip install --upgrade pip' command.
Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 7.2 MB 44.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 54.5 MB/s eta 0:00:01
     |████████████████████████████████| 756 kB 20.1 MB/s eta 0:00:01
     |████████████████████████████████| 268 kB 59.6 MB/s eta 0:00:01
     |████████████████████████████████| 7.8 MB 25.9 MB/s eta 0:00:01
     |████████████████████████████████| 139 kB 20.0 MB/s eta 0:00:01
You should c

In [2]:
!pip install bert-base==0.0.7 -i https://pypi.python.org/simple

Looking in indexes: https://pypi.python.org/simple
     |████████████████████████████████| 110 kB 132 kB/s eta 0:00:01
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7394 sha256=f14f66fbf91f8f5d3862ba0ed1a85058f3803bd6227774759e491d0b89e0509f
  Stored in directory: /home/ma-user/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil
You should consider upgrading via the '/home/ma-user/anaconda3/envs/PyTorch-1.8/bin/python3.7 -m pip install --upgrade pip' command.


In [45]:
pwd

'/home/ma-user/work/Package'

切换至Baseline里的文件目录

In [48]:
%cd Baseline/model

/home/ma-user/work/Package/Baseline/model


### 2.构建词典<br>
须在代码中替换训练集文件目录（示例：data/train.txt）<br>
生成的.json文件自动保存在vocab目录(示例：‘vocab/sent_vocab.json vocab/tag_vocab.json’)

In [95]:
!python utils.py
!python vocab.py  data/train.txt  vocab/sent_vocab.json  vocab/tag_vocab.json

The number of sentences in train.txt: 25740


### 3.模型训练<br>
修改epoch数量可以通过修改代码中('--max-epoch=10')中的数字进行修改，如未使用gpu，删除最后的('--cuda')即可，其他具体参数可在run.py查看<br>
注意训练集文件目录

In [97]:
!python run.py train data/train.txt vocab/sent_vocab.json vocab/tag_vocab.json  --dropout-rate=0.5 --lr=0.01 --lr-decay=0.8 --max-epoch=300 --batch-size=128 --cuda

/home/ma-user/anaconda3/envs/PyTorch-1.8/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)
num of training examples: 20592
num of development examples: 5148
^C




### 4.模型测试<br>
使用上一步训练的模型在测试集判分<br>
如需修改测试集，可替换test.txt，或修改.sh脚本中路径<br>
生成结果文件result.txt，可打开查看格式示例

In [65]:
!sh run.sh test

/home/ma-user/anaconda3/envs/PyTorch-1.8/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)
num of test samples: 27830
Some weights of the model checkpoint at pretrained_bert_models/minibert were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactl

In [42]:
cd ..

/home/ma-user/work/Package


### 5.模型上传至ModelArts-AI应用
将训练好的模型导入ModelArts 将模型导入ModelArts，为后续推理测试、模型提交做准备。最后显示“所有模型导入完成”证明运行成功。

In [46]:
from modelarts.session import Session
from modelarts.model import Model
from modelarts.config.model_config import TransformerConfig,Params
!pip install json5
import json5
import re
import traceback
import random
import moxing as mox

try:
    session = Session()
    config_path = 'Baseline/model/config.json'  #注意config.json路径
    if mox.file.exists(config_path): # 判断一下是否存在配置文件，如果没有则不能导入模型
        model_location =  'Baseline/model'
        model_name = "1_modelname" #模型名称可自行定义
        load_dict = json5.loads(mox.file.read(config_path))
        model_type = load_dict['model_type']
        re_name = '_'+str(random.randint(0,1000))
        model_name += re_name
        runtime=load_dict['runtime']
        print("正在导入模型,模型名称：", model_name)
        model_instance = Model(
                     session, 
                     model_name=model_name,               # 模型名称
                     model_version="1.0.0",               # 模型版本
                      source_location_type='LOCAL_SOURCE',
                     source_location=model_location,      # 模型文件路径
                     model_type=model_type,# 模型类型
                     runtime=runtime
                     )

    print("所有模型导入完成")
except Exception as e:
    print("发生了一些问题，请看下面的报错信息：") 
    traceback.print_exc()
    print("模型导入失败")

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
You should consider upgrading via the '/home/ma-user/anaconda3/envs/PyTorch-1.8/bin/python3.7 -m pip install --upgrade pip' command.
正在导入模型,模型名称： 1_modelname_883
modelarts-cn-north-4-280a6fa6 is existed


INFO:obs:Successfully upload file /home/ma-user/work/Package/Baseline/model to OBS modelarts-cn-north-4-280a6fa6/model-1206-101730
INFO:obs:Successfully upload file /home/ma-user/work/Package/Baseline/model/__pycache__ to OBS modelarts-cn-north-4-280a6fa6/model-1206-101730/model
INFO:obs:Successfully upload file /home/ma-user/work/Package/Baseline/model/__pycache__ to OBS modelarts-cn-north-4-280a6fa6/model-1206-101730/model
INFO:obs:Successfully upload file /home/ma-user/work/Package/Baseline/model/__pycache__ to OBS modelarts-cn-north-4-280a6fa6/model-1206-101730/model
INFO:obs:Successfully upload file /home/ma-user/work/Package/Baseline/model/__pycache__ to OBS modelarts-cn-north-4-280a6fa6/model-1206-101730/model
INFO:obs:Successfully upload file /home/ma-user/work/Package/Baseline/model to OBS modelarts-cn-north-4-280a6fa6/model-1206-101730
INFO:obs:Successfully upload file /home/ma-user/work/Package/Baseline/model to OBS modelarts-cn-north-4-280a6fa6/model-1206-101730
INFO:obs:Su

Successfully upload file /home/ma-user/work/Package/Baseline/model to OBS modelarts-cn-north-4-280a6fa6/model-1206-101730
Successfully upload model files from /home/ma-user/work/Package/Baseline/model to obs path /modelarts-cn-north-4-280a6fa6/model-1206-101730.
The model source location is https://modelarts-cn-north-4-280a6fa6.obs.cn-north-4.myhuaweicloud.com/model-1206-101730/model
publishing
published
所有模型导入完成
